## Featurize SLAP reactions

Added features:

- unbalanced, atom-mapped reactions without stereochemistry on the central ring (reactionSMILES)
- imine intermediates (SMILES)


In [ ]:
from IPython.display import SVG, display

import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, MolToSmiles, MolFromSmiles, rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D, MolsToGridImage

In [ ]:
# adapted from https://gist.github.com/greglandrum/61c1e751b453c623838759609dc41ef1
def draw_mol(mol, font_scale=0.8):
    """Draw mols in a nice fashion with atom indices and optional stereocenter highlighting"""
    moveAtomIdxToNotes(mol)
    d2d = rdMolDraw2D.MolDraw2DSVG(800,300)
    d2d.drawOptions().annotationFontScale=font_scale
    d2d.DrawMolecule(mol, highlightAtoms=[i[0] for i in Chem.FindMolChiralCenters(mol)])
    d2d.FinishDrawing()
    return d2d.GetDrawingText()

def moveAtomIdxToNotes(m):
    """Move atom indices to be annotations (so they can be drawn)"""
    for at in m.GetAtoms():
        if at.GetIdx():
            at.SetProp("atomNote",str(at.GetIdx()))

In [ ]:
# https://gist.github.com/greglandrum/61c1e751b453c623838759609dc41ef1
def draw_chemical_reaction(smiles, highlightByReactant=True, font_scale=1.5):
    """Draw reactions in a nice fashion with atom map numbers and optional reactant highlighting"""
    rxn = rdChemReactions.ReactionFromSmarts(smiles,useSmiles=True)
    trxn = rdChemReactions.ChemicalReaction(rxn)
    for m in trxn.GetReactants():
        moveAtomMapsToNotes(m)
    for m in trxn.GetProducts():
        moveAtomMapsToNotes(m)
    d2d = rdMolDraw2D.MolDraw2DSVG(800,300)
    d2d.drawOptions().annotationFontScale=font_scale
    d2d.DrawReaction(trxn,highlightByReactant=highlightByReactant)
    d2d.FinishDrawing()
    return d2d.GetDrawingText()

def moveAtomMapsToNotes(m):
    """Move atom maps to be annotations (so they can be drawn)"""
    for at in m.GetAtoms():
        if at.GetAtomMapNum():
            at.SetProp("atomNote",str(at.GetAtomMapNum()))

In [ ]:
df = pd.read_csv('../data/Data S2.csv')
df

In [ ]:
# define reactions
rxn_slap = AllChem.ReactionFromSmarts(   
    '[#6:1]-[#6H:2]=O.[#6:3]-[#6:4](-[#7:5])-[#6:6]-[#8,#7:7]-[#6:8]-[#14]>>[#6:1]-[#6:2]-1-[#6:8]-[#8,#7:7]-[#6:6]-[#6:4](-[#6:3])-[#7:5]-1'
)

rxn_slap_ketone_reagent = AllChem.ReactionFromSmarts(
    '[#6:1]-[#6:2]=O.[#6:3]-[#6:4](-[#7:5])-[#6:6]-[#8:7]-[#6:8]-[#14]>>[#6:1]-[#6:2]-1-[#6:8]-[#8:7]-[#6:6]-[#6:4](-[#6:3])-[#7:5]-1'
)

In [ ]:
# preprocess reactions
rxn_slap.Initialize()
AllChem.SanitizeRxn(rxn_slap)
rxn_slap.Initialize()
AllChem.SanitizeRxn(rxn_slap_ketone_reagent)

In [ ]:
# show the reaction
rxn_slap

In [ ]:
# show the reaction
rxn_slap_ketone_reagent

In [ ]:
df

In [ ]:
# test: show the first product
prod = rxn_slap.RunReactants((MolFromSmiles(df.at[1, 'Aldehyde 2']), MolFromSmiles(df.at[1, 'SLAP'])))[0][0]
prod

In [ ]:
def create_reaction_instance(rxn, reactants):
    """
    Create an instance of a reaction, given reactants, and map all atoms that end up in the product(s).
    This is adapted from Greg's code in http://github.com/rdkit/rdkit/issues/1269#issuecomment-275228746,
    but extended to map the side chains as well.
    Note that atoms that are not present in the product (unbalanced reaction equation) will not be annotated.
    """
    
    # first, we set a tag on reactant atoms. This will be passed on to the product for all non-mapped atoms
    for i, sm in enumerate(reactants):
        for atom in sm.GetAtoms():
            atom.SetProp('tag', "reactant-%s atom-%s"%(i, atom.GetIdx()))
    
    # for the mapped atoms, extract their mapping in the reactants
    map_number_to_reactant = {}
    for i,reactant in enumerate(rxn.GetReactants()):
        for atom in reactant.GetAtoms():
            if atom.HasProp('molAtomMapNumber'):
                map_number_to_reactant[atom.GetIntProp('molAtomMapNumber')] = (i,atom.GetIdx())
    
    mapped_reactions = []  # this will hold the reactions 
    product_set = rxn.RunReactants(reactants)  # run the reaction to get product set
    
    # now, we look into the products
    for products in product_set:
        # we need to know the highest mapno, because mapping the "tagged" atoms will have to start above that
        mapno_max = max(map_number_to_reactant.keys())  # needs to reset on every product_set
        reactant_list = [Chem.Mol(x) for x in reactants]
        reaction = AllChem.ChemicalReaction()
        for p in products:
            for atom in p.GetAtoms():
                
                # for atoms that are mapped in the reaction template
                if atom.HasProp('old_mapno'):
                    mno = atom.GetIntProp('old_mapno')
                    atom.SetIntProp('molAtomMapNumber',mno)
                    ridx,aidx = map_number_to_reactant[mno] 
                    # aidx is the index of the atom in the reactant template. We need
                    # to read out the number in the actual reactant:
                    raidx = int(atom.GetProp("react_atom_idx"))
                    ratom = reactant_list[ridx].GetAtomWithIdx(raidx).SetIntProp('molAtomMapNumber',mno)
                    
                # for atoms that are unmapped in the reaction template
                elif atom.HasProp('tag'):
                    tag = atom.GetProp('tag')
                    mapno_max += 1
                    atom.SetIntProp('molAtomMapNumber', mapno_max)
                    # now find the tag in reactant_list
                    for sm in reactant_list:
                        for ratom in sm.GetAtoms():
                            if ratom.HasProp('tag'):
                                if ratom.GetProp('tag') == tag:
                                    ratom.SetIntProp('molAtomMapNumber', mapno_max)
                                    
            # now add the product to the reaction
            reaction.AddProductTemplate(p)
        # add the reactants to reaction    
        for reactant in reactant_list:
            reaction.AddReactantTemplate(reactant)
        # add reaction for all product sets
        mapped_reactions.append(reaction)
    return mapped_reactions


def map_reactions(rxn, reactant_sets):
    """Take a reaction template and a list of reactant sets and return the mapped reactions."""
    ketone_slap_substructure = MolFromSmiles('NC(COC[Si](C)(C)C)(C)C')
    mapped_reactions = []
    for i, reactant_set in enumerate(reactant_sets):
        reaction_inst = create_reaction_instance(rxn, reactant_set)
        if len(reaction_inst) == 1:  # all good
            mapped_reactions.append(reaction_inst[0])
        elif len(reaction_inst) == 0:  # failed
            mapped_reactions.append(None)
            print(f'ERROR: No product for reactant set with index {i}')
        elif len(reaction_inst) == 2 and reaction_inst[0].GetReactants()[1].HasSubstructMatch(ketone_slap_substructure): # it's a ketone so it will give two products
            # ketone SLAP products may or may not be identical, depending on whether the starting ketone was assymetric
            # to compare the smiles, we need to remove molAtomMapNumbers. We need to work on copies to not clear them from the reaction instance
            products = [Chem.Mol(reaction_inst[0].GetProducts()[0]), Chem.Mol(reaction_inst[1].GetProducts()[0])]
            for p in products:
                for atom in p.GetAtoms():
                    if atom.HasProp('molAtomMapNumber'):
                        atom.ClearProp('molAtomMapNumber')
            if MolToSmiles(products[0]) == MolToSmiles(products[1]):  # products are identical and we can discard one reaction
                mapped_reactions.append(reaction_inst[0])
            else:
                print(f'WARNING: Multiple stereoisomeric products for reactant set with index {i}')
                mapped_reactions.append(reaction_inst)
        else:  # failed
            mapped_reactions.append(reaction_inst)
            print(f'ERROR: Multiple products for reactant set with index {i}')
    return mapped_reactions
        

In [ ]:
# add mols to dataframe
df["Aldehyde 2 MOL"] = df["Aldehyde 2"].apply(MolFromSmiles)
df["SLAP MOL"] = df["SLAP"].apply(MolFromSmiles)

In [ ]:
df.at[1151, "SLAP MOL"]

In [ ]:
reactions = map_reactions(rxn_slap, df[['Aldehyde 2 MOL', 'SLAP MOL']].values.tolist())

In [ ]:
len(reactions)

In [ ]:
reactions[192]

In [ ]:
# ensure that there are no lists of reactions (if something had given multiple products)
for reac in reactions:
    assert type(reac) is rdChemReactions.ChemicalReaction

In [ ]:
# generate reactionSMILES
reaction_smiles = []
for reac in reactions:
    reaction_smiles.append(Chem.AllChem.ReactionToSmiles(reac))
len(reaction_smiles)

In [ ]:
# check one
display(SVG(draw_chemical_reaction(reaction_smiles[250])))

In [ ]:
# now add the reaction smiles to the dataframe
df['reactionSMILES'] = reaction_smiles

### Intermediate imine
We want to have one more option for featurization:
It is possible to use the intermediate imine as the only feature because it contains all the information about the reaction.

In [ ]:
reaction_to_imine = AllChem.ReactionFromSmarts(
        "[#6:1]-[#6H1:2]=O.[#6:3]-[#6:4](-[#7:5])-[#6:6]-[#8,#7:7]-[#6:8]-[#14:9]>>[#6:1]\[#6:2]=[#7:5]\[#6:4](-[#6:3])-[#6:6]-[#8,#7:7]-[#6:8]-[#14:9]"
    )


In [ ]:
# preprocess reaction
reaction_to_imine.Initialize()
AllChem.SanitizeRxn(reaction_to_imine)

In [ ]:
reaction_to_imine

In [ ]:
# we just use the machinery we have built above and grab the product / remove the atom-labels later
reactions = map_reactions(reaction_to_imine, df[['Aldehyde 2 MOL', 'SLAP MOL']].values.tolist())

In [ ]:
# grab product
imines = [r.GetProducts()[0] for r in reactions]
len(imines)

In [ ]:
# remove atom mapping
for imine in imines:
    AllChem.SanitizeMol(imine)
    for atom in imine.GetAtoms():
        if atom.HasProp("molAtomMapNumber"):
            atom.ClearProp("molAtomMapNumber")

In [ ]:
MolsToGridImage(imines[0:16])

In [ ]:
# add imines to df
df["imines"] = [MolToSmiles(i) for i in imines]

In [ ]:
# remove mols before saving
df = df.drop(columns=["Aldehyde 2 MOL", "SLAP MOL"])
# save to file
df.to_csv('../data/featurized_dataS2.csv', index=False)